<a href="https://colab.research.google.com/github/EtzionR/LM4GeoAI/blob/main/Geo_LLM_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ex 3: Geo LLM Agents

### created by Etzion Harari | Geo-AI Course

[**https://github.com/EtzionR/LM4GeoAI**](https://github.com/EtzionR/LM4GeoAI)

In [1]:
!pip install wikipedia
!pip install cohere
!pip install osmnx


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=dcf08df0c6ab210017df0d697722a93ed58b857719693bf562e1dfebe8a5beef
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.0/319.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 6.8 MB/s eta 0:00:00


In [2]:
from difflib import get_close_matches
from getpass import getpass

import numpy as np
import osmnx as ox
import wikipedia
import folium
import cohere

Cohere models: [https://docs.cohere.com/docs/models#command](https://docs.cohere.com/docs/models#command)

Cohere MCP: [https://docs.cohere.com/page/basic-tool-use](https://docs.cohere.com/page/basic-tool-use)

In [3]:
MODEL  = "command-a-03-2025"
TEMP = .01

Get your Cohere API Key:

[https://dashboard.cohere.com/api-keys](https://dashboard.cohere.com/api-keys)

In [4]:
KEY = getpass("Please Enter COHERE KEY:\n")

len(KEY)

Please Enter COHERE KEY:
··········


40

In [5]:
client = cohere.Client(api_key=KEY)
client

In [6]:
QUERY = 'What is panda bear?'

output = client.chat(model=MODEL,
                     message=QUERY,
                     temperature=TEMP).text

print(output)

A panda bear, commonly known as the giant panda, is a bear native to central China. It is easily recognized by its distinctive black and white coat, with black patches around the eyes, ears, and across its body. Giant pandas are primarily herbivores, and their diet consists mainly of bamboo, though they may also eat other plants, small animals, and fish.

Giant pandas are an iconic symbol of conservation efforts. They are classified as **Vulnerable** on the IUCN Red List, with their population threatened by habitat loss, poaching, and low reproductive rates. Conservation programs, including breeding initiatives and habitat protection, have helped stabilize and slightly increase their numbers in recent years.

Pandas are solitary animals and are known for their gentle nature. They spend much of their day eating, as bamboo is low in nutrients and requires large quantities to meet their energy needs. They are also excellent climbers and can swim to escape predators or find food.

The gian

In [7]:
def get_pois_near_adress(location_adress, place_type, radius=500):
    """
    Load OSM POIs from given place_type

    :location_adress: location adress for searching OSM
    :place_type: type of point of interest (POI)
    :radius: radius around location_adress (in meters, int, default: 500)

    :return: selected dataframe
    """

    # Load data from OSM
    out = ox.features_from_address(location_adress, {"amenity": True}, radius)
    out = out[['name','amenity','geometry']]
    out = out.rename(columns={'amenity':'place_type'})

    # Find the closets place_type (using SequenceMatcher)
    place_type = get_close_matches(place_type,
                                   [*out.place_type.unique()],
                                   n=1,
                                   cutoff=.0)[0]

    # select the place_type
    dataframe = out[out.place_type==place_type]

    return dataframe

# ------------------------------------------------------------------------------


# get_place_info application:

location_adress = 'מגדלי עזריאלי'
place_type      = 'fual'           # not fuel!

get_pois_near_adress(location_adress, place_type)

name place_type                   geometry
element id                                                  
node    442893138  דלק       fuel  POINT (34.79586 32.07468)
        442893144  דלק       fuel  POINT (34.79449 32.07144)
        442898671   פז       fuel   POINT (34.79724 32.0719)

In [8]:
def shortest_path_between_adresses(adress_1, adress_2,  gap=.25):
    """
    Calculate shortest path between two adresses using OSM roads network

    :adress_1: origin adress (source)
    :adress_2: destination adress (target)
    :gap: buffer ratio from bbox (float, default: .25)

    :return: shortest path between adress_1 to adress_2 (coordinates)
    """

    # gecode to both adresses
    y1, x1 = ox.geocode(adress_1)
    y2, x2 = ox.geocode(adress_2)

    # build the bbox
    delta = max(abs(x1-x2)*gap, abs(y1-y2)*gap)
    bbox = [min(x1,x2)-delta, min(y1,y2)-delta, max(x1,x2)+delta, max(y1,y2)+delta]

    # get the roads network graph
    G = ox.graph.graph_from_bbox(bbox, truncate_by_edge =True, network_type ='drive', simplify =False)

    # find the origin & destination nodes
    node1 = ox.distance.nearest_nodes(G, x1, y1)
    node2 = ox.distance.nearest_nodes(G, x2, y2)

    # calculate path between origin to destination
    route = ox.routing.shortest_path(G, orig=node1, dest=node2)
    route_xy = [(G.nodes[osmid]['x'], G.nodes[osmid]['y']) for osmid in route]

    return route_xy

# ------------------------------------------------------------------------------


# shortest_path_between_adresses application:

ad1 = "אוניברסיטת תל אביב"
ad2 = 'מגדלור רידינג'

path = shortest_path_between_adresses(ad1, ad2)

print(f'Number of nodes in the output path: {len(path)}\n\nFirst 5 points:\n{path[:5]}')

Number of nodes in the output path: 128

First 5 points:
[(34.8037004, 32.1107766), (34.8037061, 32.1107118), (34.8037047, 32.1105204), (34.8036907, 32.1103309), (34.8036844, 32.1101144)]


In [9]:
system_prompt = """You are an AI assistant with access to the following tools:
get_pois_near_adress tool, which enable you to run osm query.
shortest_path_between_adresses tool, which enable you to calculate route between to given place names."""

tools = [
    {
        "name": "get_pois_near_adress",
        "description": "can return every type of geographic location near some given adress",
        "parameter_definitions": {
            "location_adress": {
                "description": "free text location adress",
                "type": "str",
                "required": True
            },
            "place_type": {
                "description": "type of point of interest (POI)",
                "type": "str",
                "required": True
            },
            "radius": {
                "description": "radius of search in meters",
                "type": "float",
                "required": False
            }
        }
    },

    {
        "name": "shortest_path_between_adresses",
        "description": "Calculate shortest path between two adresses using OSM roads network",
        "parameter_definitions": {
            "adress_1": {
                "description": "adress of the origin point (free text)",
                "type": "str",
                "required": True
            },
            "adress_2": {
                "description": "adress of the destination point (free text)",
                "type": "str",
                "required": True
            },
            "gap": {
                "description": "buffer ratio of the bbox to load roads network",
                "type": "float",
                "required": False
            }
        }
    },
]

function_maps = {get_pois_near_adress.__name__: get_pois_near_adress,
                 shortest_path_between_adresses.__name__: shortest_path_between_adresses}
function_maps

{'get_pois_near_adress': <function __main__.get_pois_near_adress(location_adress, place_type, radius=500)>,
 'shortest_path_between_adresses': <function __main__.shortest_path_between_adresses(adress_1, adress_2, gap=0.25)>}

In [10]:
HEB = 'מגדלי עזריאלי'

user_prompt = f"I want you to return every cafe near {HEB}"


response = client.chat(
    model = MODEL,
    message = user_prompt,
    preamble = system_prompt,
    tools = tools
)

print(f"Model output: {response.text}\n\n")

out = function_maps[response.tool_calls[0].name](**response.tool_calls[0].parameters)
out

Model output: I will use the get_pois_near_adress tool to find every cafe near מגדלי עזריאלי.




name place_type                   geometry
element id                                                            
node    553022802       קפה שרגא       cafe    POINT (34.7903 32.0707)
        1207157419           Uno       cafe  POINT (34.78908 32.07748)
        3465156312        רולדין       cafe  POINT (34.78693 32.07173)
        6469411087        לנדוור       cafe  POINT (34.79164 32.07452)
        6469411088       קפה קפה       cafe   POINT (34.79172 32.0747)
        6644165508         Aroma       cafe  POINT (34.78769 32.07308)
        7211406506   Max Brenner       cafe   POINT (34.78677 32.0715)
        10784964980      Arcaffe       cafe  POINT (34.78864 32.07829)
        10812416453       BEITEA       cafe  POINT (34.78798 32.07223)
        11802167414      Arcaffe       cafe  POINT (34.79304 32.07731)
        12032015683          mae       cafe  POINT (34.78829 32.07195)
        12139988888     קפה עוטף       cafe  POINT (34.78739 32.07308)
        12635329119       רולדין       cafe   POINT (34.79232 32.0743)
        13338979218         Biga       cafe  POINT (34.78691 32.07308)

In [11]:
fmap = folium.Map(location=[out.geometry.y.median(), out.geometry.x.median()],
                  zoom_start=15.5, width=500, height=500)

folium.GeoJson(out).add_to(fmap)

fmap

In [12]:
user_prompt = f"אני רוצה לדעת מה המסלול מחוף מציצים למגדלי עזריאלי"


response = client.chat(
    model = MODEL,
    message = user_prompt,
    preamble = system_prompt,
    tools = tools
)

print(f"Model output: {response.text}\n\n")

out = function_maps[response.tool_calls[0].name](**response.tool_calls[0].parameters)

print(f'Output length: {len(out)}')

Model output: אשתמש בכלי shortest_path_between_adresses כדי למצוא את המסלול הקצר ביותר מחוף מציצים למגדלי עזריאלי.


Output length: 137


In [13]:
import numpy as np

fmap = folium.Map(location=[np.array(out)[:,1].mean(), np.array(out)[:,0].mean()],
                  zoom_start=14, width=500, height=500)

folium.PolyLine(
    locations=[(y,x) for x,y in out],
    color="blue",
    weight=5,
    tooltip=user_prompt,
).add_to(fmap)

fmap

# -----------------------------------------------------------------------

In [14]:


# search by query (not exact match)
results = wikipedia.search("machine learning in healthcare")
print(results)

# pick the first match
page = wikipedia.page(results[0])
print(page.title)
print(page.content)

['Artificial intelligence in healthcare', 'Machine learning', 'Health informatics', 'Explainable artificial intelligence', 'Deep reinforcement learning', 'Suchi Saria', 'Deep learning', 'Political methodology', 'Ensemble learning', 'Danielle Belgrave']
Artificial intelligence in healthcare
Artificial intelligence in healthcare is the application of artificial intelligence (AI) to analyze and understand complex medical and healthcare data. In some cases, it can exceed or augment human capabilities by providing better or faster ways to diagnose, treat, or prevent disease.
As the widespread use of artificial intelligence in healthcare is still relatively new, research is ongoing into its applications across various medical subdisciplines and related industries. AI programs are being applied to practices such as diagnostics, treatment protocol development, drug development, personalized medicine, and patient monitoring and care. Since radiographs are the most commonly performed imaging tes

In [34]:
point = (38.8893845,-77.0352332)

options = ox.features.features_from_point(center_point = point, dist=100, tags = {'building': True, "amenity": True})
options = options[['geometry','name']].dropna().drop_duplicates()

options

,,geometry,name
element,id,,
way,766761337,"POLYGON ((-77.03514 38.88952, -77.03514 38.889...",Washington Monument


In [35]:
sorted([*options.name])

['Washington Monument']

In [36]:


WGS84_UTM_18N = "EPSG:32618"

options.to_crs(WGS84_UTM_18N)

,,geometry,name
element,id,,
way,766761337,"POLYGON ((323491.592 4306484.783, 323491.423 4...",Washington Monument


In [37]:


results = wikipedia.search('Washington Monument')
print(results)

# pick the first match
page = wikipedia.page(results[0])
print(page.title)
print(page.content)

['Washington Monument', 'Washington Monument (Baltimore)', 'George Washington Birthplace National Monument', 'Washington Monument (disambiguation)', 'Washington Monument (Philadelphia)', 'Virginia Washington Monument', 'Washington Monument syndrome', 'George Washington', 'Monument', 'List of memorials to George Washington']
Washington Monument
The Washington Monument is a 555-foot (169 m) tall obelisk on the National Mall in Washington, D.C., built to commemorate George Washington, a Founding Father of the United States and the nation's first president. Standing east of the Reflecting Pool and the Lincoln Memorial, the monument is made of bluestone gneiss for the foundation and of granite for the construction. The outside facing consists of three different kinds of white marble, as the building process was repeatedly interrupted. The monument stands 554 feet 7+11⁄32 inches (169.046 m) tall, according to U.S. National Geodetic Survey measurements in 2013 and 2014.  It is the third talle